In [8]:
import numpy as np
import dask.array as da
from dask.distributed import Client, progress
import coiled

In [7]:
import sys
sys.path.append('..')

In [9]:
from utils.parallel import apply_along_axis_parallel

In [3]:
cluster = coiled.Cluster(
    n_workers=10,
    region="us-east-1",
)
client = cluster.get_client()
client

Output()

╭────────────────────────── Not Synced with Cluster ───────────────────────────╮
│                  ╷                                               ╷           │
│   Package        │ Error                                         │ Risk      │
│ ╶────────────────┼───────────────────────────────────────────────┼─────────╴ │
│   stein-thinning │ cannot find stein-thinning~=1.1.0 on          │ Warning   │
│                  │ pypi.org. If you are using a custom PyPI URL, │           │
│                  │ ensure it is set by running                   │           │
│                  │   pip config set global.extra-index-url <url> │           │
│                  │ (replacing <url> with your custom PyPI URL).  │           │
│                  │                                               │           │
│                  ╵                                               ╵           │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

/home/artem/Study/KCL/ST50 Project/code/.venv/lib/python3.11/site-packages/distributed/client.py:1394: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | None   | 4.3.3     | 4.3.3   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tls://10.0.173.29:8786' processes=8 threads=32, memory=118.67 GiB>

In [12]:
rng = np.random.default_rng(12345)
x = rng.random((10_000, 10_000))
x

array([[0.22733602, 0.31675834, 0.79736546, ..., 0.02125837, 0.57321308,
        0.5506928 ],
       [0.92367914, 0.89759528, 0.67357437, ..., 0.91144091, 0.24277272,
        0.51630166],
       [0.8394219 , 0.25178813, 0.65764543, ..., 0.73324734, 0.14734864,
        0.11094437],
       ...,
       [0.55684932, 0.9160493 , 0.60178455, ..., 0.07347207, 0.75078319,
        0.22758357],
       [0.32300271, 0.17848871, 0.9127623 , ..., 0.02110224, 0.51982813,
        0.03280058],
       [0.13569191, 0.66540941, 0.07010956, ..., 0.57398387, 0.7631287 ,
        0.29704948]])

In [15]:
%%time
np.apply_along_axis(np.sum, 1, x)

CPU times: user 185 ms, sys: 0 ns, total: 185 ms
Wall time: 187 ms


array([4959.45131033, 4991.39413747, 4981.93167539, ..., 4968.39877292,
       4989.88007011, 5006.02789202])

In [17]:
%%time
apply_along_axis_parallel(np.sum, 1, x, 500, client)

/home/artem/Study/KCL/ST50 Project/code/.venv/lib/python3.11/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 38.15 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 2.78 s, sys: 757 ms, total: 3.54 s
Wall time: 24.9 s


array([4959.45131033, 4991.39413747, 4981.93167539, ..., 4968.39877292,
       4989.88007011, 5006.02789202])